In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
from sklearn.model_selection import KFold

In [2]:
def convert_cat_to_num(encoder, data):
	encoder.fit(data)
	return encoder.transform(data)

def convert_num_to_cat(encoder, data):
	return encoder.inverse_transform(data)

In [3]:
!unzip HW4_q2.zip
!cd HW4_q2

Archive:  HW4_q2.zip
   creating: HW4_q2/
  inflating: HW4_q2/adult.data       
  inflating: HW4_q2/adult.names      
  inflating: HW4_q2/adult.test       


In [4]:
cd HW4_q2/

/content/HW4_q2


In [5]:
testFile = open('adult.test')
namesFile = open('adult.names')
trainFile = open('adult.data')

testData = np.genfromtxt(testFile, delimiter=',', dtype=np.str_)
trainData = np.genfromtxt(trainFile, delimiter=',', dtype=np.str_)
#namesData = np.genfromtxt(namesFile, delimiter=',', skip_header=96, dtype=np.str_)

splitter = trainData.shape[0]
combinedData = np.concatenate((trainData, testData), axis=0)

cat_list=[0,1,0,1,0,1,1,1,1,1,0,0,0,1,1]
enc = []
for i in range(len(cat_list)):
  enc.append(OrdinalEncoder())

for i in range(len(cat_list)):
  if (cat_list[i]==1):
    combinedData[:,i:i+1] = convert_cat_to_num(enc[i], combinedData[:,i:i+1])


X_train = combinedData[:splitter][:,:-1]
y_train = combinedData[:splitter][:,-1:]

X_test = combinedData[splitter:][:,:-1]
y_test = combinedData[splitter:][:,-1:]
# XX = np.concatenate((X_train, X_test), axis=0)
# yy = np.concatenate((y_train, y_test), axis=0)

print(X_test)
print(y_test)

print(X_test[0])
print(trainData[0])
model = XGBClassifier()
model.fit(X_train, y_train)

[['25' '4.0' ' 226802' ... ' 0' ' 40' '39.0']
 ['38' '4.0' ' 89814' ... ' 0' ' 50' '39.0']
 ['28' '2.0' ' 336951' ... ' 0' ' 40' '39.0']
 ...
 ['38' '4.0' ' 374983' ... ' 0' ' 50' '39.0']
 ['44' '4.0' ' 83891' ... ' 0' ' 40' '39.0']
 ['35' '5.0' ' 182148' ... ' 0' ' 60' '39.0']]
[['0.0']
 ['0.0']
 ['1.0']
 ...
 ['0.0']
 ['0.0']
 ['1.0']]
['25' '4.0' ' 226802' '1.0' ' 7' '4.0' '7.0' '3.0' '2.0' '1.0' ' 0' ' 0'
 ' 40' '39.0']
['39' ' State-gov' ' 77516' ' Bachelors' ' 13' ' Never-married'
 ' Adm-clerical' ' Not-in-family' ' White' ' Male' ' 2174' ' 0' ' 40'
 ' United-States' ' <=50K']


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [6]:
y_pred = model.predict(X_train)
print(accuracy_score(y_train, y_pred))
print(confusion_matrix(y_train, y_pred))

y_pred = model.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

0.8665274408034151
[[23560  1160]
 [ 3186  4655]]
0.8661630120999939
[[11857   578]
 [ 1601  2245]]


In [8]:
search_params = [
    (max_depth, min_child_weight)
    for max_depth in range(9,12)
    for min_child_weight in range(5,8)
]

In [11]:
#modelk = XGBClassifier()
kf = KFold(n_splits=10)
kf.get_n_splits(X_train)
best_max_depth = 0
best_min_child_weight = 0
best_accuracy_score = 0
for max_depth, min_child_weight in search_params:
  params = {}
  params['max_depth'] = max_depth
  params['min_child_weight'] = min_child_weight
  acs = []
  for train_index, test_index in kf.split(X_train):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_traink, X_testk = X_train[train_index], X_train[test_index]
    y_traink, y_testk = y_train[train_index], y_train[test_index]
    #dtrain = np.concatenate((X_traink, y_traink),axis=1)
    modelk = XGBClassifier(**params)
    modelk.fit(X_traink, y_traink)
    y_pred = modelk.predict(X_testk)
    acs.append(accuracy_score(y_testk, y_pred))
    #print(acs[-1])
  if best_accuracy_score < np.mean(acs):
    best_accuracy_score = np.mean(acs)
    print("best " , best_accuracy_score)
    best_max_depth = max_depth
    best_min_child_weight = min_child_weight
    #print(confusion_matrix(y_testk, y_pred))


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8664415105925698
0.8753071253071253
0.8805282555282555
0.8633292383292384
0.8728501228501229
0.8725429975429976
0.8716216216216216
0.8768427518427518
0.8737714987714987
0.8703931203931204
best  0.8723628242779302


In [12]:
#got the best params from k-fold
params = {}
params['max_depth'] = best_max_depth
params['min_child_weight'] = best_min_child_weight
print(params)

modelf = XGBClassifier(**params)
modelf.fit(X_train, y_train)

y_pred = modelf.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


{'max_depth': 9, 'min_child_weight': 7}


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.872366562250476
[[11701   734]
 [ 1344  2502]]
